In [0]:
import json
json_data=[]
for line in open('/content/drive/My Drive/BigDataA1/Software_5.json', 'r'):
  json_data.append(json.loads(line))

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://ftp.wayne.edu/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("/content/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

sc = spark.sparkContext

--2020-05-18 23:00:01--  http://ftp.wayne.edu/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
Resolving ftp.wayne.edu (ftp.wayne.edu)... 141.217.0.199
Connecting to ftp.wayne.edu (ftp.wayne.edu)|141.217.0.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232530699 (222M) [application/x-gzip]
Saving to: ‘spark-2.4.5-bin-hadoop2.7.tgz’

spark-2.4.5-bin-had 100%[===================>] 221.76M  27.8MB/s    in 14s     

2020-05-18 23:00:15 (16.3 MB/s) - ‘spark-2.4.5-bin-hadoop2.7.tgz’ saved [232530699/232530699]



In [0]:
import json
import datetime

def filterfunc(record):
  nrec=json.loads(record)
  return (nrec['asin']+nrec['reviewerID'],[(nrec['asin'],nrec['reviewerID'],nrec['overall'],nrec['reviewTime'])])

def remDuplicates(records):
  records=records[1]
  latest=None
  toret=None
  for rec in records:
    curr=datetime.datetime.strptime(rec[-1],'%m %d, %Y')
    if not latest:
      latest=curr
      toret=rec
    elif curr>latest:
      latest=curr
      toret=rec
  return [toret]


In [0]:
iprdd=sc.textFile('/content/drive/My Drive/BigDataA1/Software_5.json')

In [0]:
rdd1=iprdd.map(lambda x: filterfunc(x)).reduceByKey(lambda a,b:a+b).map(lambda x:remDuplicates(x))

In [0]:
rdd1.take(5)

[[('0321719816', 'ACJT8MUC0LRF0', 5.0, '10 16, 2010')],
 [('0321719816', 'A2BVNVJOFXGZUB', 5.0, '09 26, 2010')],
 [('0321719816', 'A2JMJVNTBL7K7E', 5.0, '04 7, 2011')],
 [('0321719816', 'A2WCFDOCS73PNB', 5.0, '10 28, 2010')],
 [('0321719816', 'A2VWE5SGNDS8HW', 3.0, '10 21, 2010')]]

In [0]:
# rdd2=rdd1.map(lambda x:(x[0][0],x)).reduceByKey(lambda a,b: a+b).filter(lambda x: len(x[1])>=25).flatMap(lambda x: x[1]).map(lambda x: [x])
rdd2=rdd1.map(lambda x:(x[0][0],x)).reduceByKey(lambda a,b: a+b).flatMap(lambda x: x[1]).map(lambda x: [x])

In [0]:
# rdd3=rdd2.map(lambda x: (x[0][1],x)).reduceByKey(lambda a,b: a+b).filter(lambda x: len(x[1])>=10).flatMap(lambda x: x[1]).map(lambda x: [x])
rdd3=rdd2.map(lambda x: (x[0][1],x)).reduceByKey(lambda a,b: a+b).flatMap(lambda x: x[1]).map(lambda x: [x[:-1]])

In [0]:
rdd3.take(3)

[[('0321719816', 'ACJT8MUC0LRF0', 5.0)],
 [('B00JHN1MJO', 'ACJT8MUC0LRF0', 5.0)],
 [('B009APQ6S6', 'ACJT8MUC0LRF0', 5.0)]]

In [0]:
rdd3.map(lambda x: (x[0][0],x)).reduceByKey(lambda a,b:a+b).take(2)

[('0321719816',
  [('0321719816', 'ACJT8MUC0LRF0', 5.0),
   ('0321719816', 'A2BVNVJOFXGZUB', 5.0),
   ('0321719816', 'A2VWE5SGNDS8HW', 3.0),
   ('0321719816', 'A2FRAS05LEI6JJ', 4.0),
   ('0321719816', 'A38NELQT98S4H8', 4.0),
   ('0321719816', 'AYUF7YETYOLNX', 5.0),
   ('0321719816', 'A31ICLWQ9CSHRS', 5.0),
   ('0321719816', 'A14JBDSWKPKTZA', 5.0),
   ('0321719816', 'A2JMJVNTBL7K7E', 5.0),
   ('0321719816', 'A2WCFDOCS73PNB', 5.0),
   ('0321719816', 'A3M1PA1C98XNGN', 4.0),
   ('0321719816', 'A3QJU4FEN8PQSZ', 4.0),
   ('0321719816', 'A14638TGYH7GD9', 5.0),
   ('0321719816', 'AMRMK86X3PKXD', 5.0)]),
 ('B00JHN1MJO',
  [('B00JHN1MJO', 'ACJT8MUC0LRF0', 5.0),
   ('B00JHN1MJO', 'A1GWG5CWLKJ7ET', 5.0),
   ('B00JHN1MJO', 'A1HOLE9R6WPT85', 5.0),
   ('B00JHN1MJO', 'A225G2TFM76GYX', 4.0),
   ('B00JHN1MJO', 'A3BVN2QBNPBHCR', 5.0),
   ('B00JHN1MJO', 'ATDE9JYCPI0L1', 4.0),
   ('B00JHN1MJO', 'A10G4BPT5MGBHY', 4.0),
   ('B00JHN1MJO', 'AG1OL1K3SFURN', 4.0),
   ('B00JHN1MJO', 'A26R7LLAFTPC3U', 4.0),
   ('B

In [0]:
['0321719816','B00JHN1MJO']


In [0]:
def getFiltered(x):
  if x[0] in ['0321719816','B00JHN1MJO']:
    return True
  return False

def makeDict(x):
  dict1={}
  count=0
  summ=0
  sq_sum=0
  for rec in x[1]:
    summ+=rec[2]
    sq_sum+=rec[2]**2
    count+=1
  mean=summ/count
  for rec in x[1]:
    dict1[rec[1]]=rec[2]-mean
  return (x[0],dict1,sq_sum**0.5)

def makeTuple(x):
  li=[]
  count=0
  summ=0
  sq_sum=0
  for rec in x[1]:
    summ+=rec[2]
    sq_sum+=rec[2]**2
    count+=1
  mean=summ/count
  for rec in x[1]:
    li.append((rec[1],rec[2]-mean))
  return (x[0],li,sq_sum**0.5)

In [0]:
rdd3.map(lambda x: (x[0][0],x)).reduceByKey(lambda a,b:a+b).filter(lambda x:not getFiltered(x)).take(3)

[('B009APQ6S6',
  [('B009APQ6S6', 'ACJT8MUC0LRF0', 5.0),
   ('B009APQ6S6', 'A225G2TFM76GYX', 5.0),
   ('B009APQ6S6', 'ARADXCOUVBS09', 4.0),
   ('B009APQ6S6', 'AAEHOTZ1GRAUX', 4.0),
   ('B009APQ6S6', 'A2LW5AL0KQ9P1M', 5.0),
   ('B009APQ6S6', 'A7H1LWFBE905R', 5.0)]),
 ('B000T9LUB0',
  [('B000T9LUB0', 'A2BVNVJOFXGZUB', 1.0),
   ('B000T9LUB0', 'A225G2TFM76GYX', 1.0),
   ('B000T9LUB0', 'A8G2W3PNBM0R2', 5.0),
   ('B000T9LUB0', 'A2QQEURPB1VQF7', 1.0),
   ('B000T9LUB0', 'A1PCYAHFDDMGJZ', 1.0),
   ('B000T9LUB0', 'A250AXLRBVYKB4', 2.0),
   ('B000T9LUB0', 'A2I3XS9TO93Q0F', 5.0),
   ('B000T9LUB0', 'A2UOHALGF2X77Q', 2.0),
   ('B000T9LUB0', 'A1H378KKY8V01U', 5.0)]),
 ('B001U3PYLQ',
  [('B001U3PYLQ', 'A2BVNVJOFXGZUB', 4.0),
   ('B001U3PYLQ', 'AJK49DENMS2BT', 1.0),
   ('B001U3PYLQ', 'AFBXQDFJF33VS', 4.0),
   ('B001U3PYLQ', 'A57Q87B0TC9IJ', 4.0),
   ('B001U3PYLQ', 'A1FRADBDXQPJJC', 5.0),
   ('B001U3PYLQ', 'A1WSFHRBY2ZD1R', 5.0),
   ('B001U3PYLQ', 'A14ENWEKTHCBXR', 5.0),
   ('B001U3PYLQ', 'A1F2EYORWOZ3G

In [0]:
recNeed=sc.broadcast(rdd3.map(lambda x: (x[0][0],x)).reduceByKey(lambda a,b:a+b).filter(lambda x:getFiltered(x)).map(lambda x:makeDict(x)).collect())
rdd4=rdd3.map(lambda x: (x[0][0],x)).reduceByKey(lambda a,b:a+b).filter(lambda x:not getFiltered(x)).map(lambda x:makeTuple(x))

In [0]:
rdd4.take(3)

[('B009APQ6S6',
  [('ACJT8MUC0LRF0', 0.33333333333333304),
   ('A225G2TFM76GYX', 0.33333333333333304),
   ('ARADXCOUVBS09', -0.666666666666667),
   ('AAEHOTZ1GRAUX', -0.666666666666667),
   ('A2LW5AL0KQ9P1M', 0.33333333333333304),
   ('A7H1LWFBE905R', 0.33333333333333304)],
  11.489125293076057),
 ('B000T9LUB0',
  [('A2BVNVJOFXGZUB', -1.5555555555555554),
   ('A225G2TFM76GYX', -1.5555555555555554),
   ('A8G2W3PNBM0R2', 2.4444444444444446),
   ('A2QQEURPB1VQF7', -1.5555555555555554),
   ('A1PCYAHFDDMGJZ', -1.5555555555555554),
   ('A250AXLRBVYKB4', -0.5555555555555554),
   ('A2I3XS9TO93Q0F', 2.4444444444444446),
   ('A2UOHALGF2X77Q', -0.5555555555555554),
   ('A1H378KKY8V01U', 2.4444444444444446)],
  9.327379053088816),
 ('B001U3PYLQ',
  [('A2BVNVJOFXGZUB', -0.08000000000000007),
   ('AJK49DENMS2BT', -3.08),
   ('AFBXQDFJF33VS', -0.08000000000000007),
   ('A57Q87B0TC9IJ', -0.08000000000000007),
   ('A1FRADBDXQPJJC', 0.9199999999999999),
   ('A1WSFHRBY2ZD1R', 0.9199999999999999),
   ('A1

In [0]:
recNeed.value

[('0321719816',
  {'A14638TGYH7GD9': 0.4285714285714288,
   'A14JBDSWKPKTZA': 0.4285714285714288,
   'A2BVNVJOFXGZUB': 0.4285714285714288,
   'A2FRAS05LEI6JJ': -0.5714285714285712,
   'A2JMJVNTBL7K7E': 0.4285714285714288,
   'A2VWE5SGNDS8HW': -1.5714285714285712,
   'A2WCFDOCS73PNB': 0.4285714285714288,
   'A31ICLWQ9CSHRS': 0.4285714285714288,
   'A38NELQT98S4H8': -0.5714285714285712,
   'A3M1PA1C98XNGN': -0.5714285714285712,
   'A3QJU4FEN8PQSZ': -0.5714285714285712,
   'ACJT8MUC0LRF0': 0.4285714285714288,
   'AMRMK86X3PKXD': 0.4285714285714288,
   'AYUF7YETYOLNX': 0.4285714285714288},
  17.26267650163207),
 ('B00JHN1MJO',
  {'A10G4BPT5MGBHY': -0.4375,
   'A1BEKAQQGP8CMF': -0.4375,
   'A1GWG5CWLKJ7ET': 0.5625,
   'A1HOLE9R6WPT85': 0.5625,
   'A1V3TRGWOMA8LC': 0.5625,
   'A225G2TFM76GYX': -0.4375,
   'A25NT9MZV9RTR7': -0.4375,
   'A26R7LLAFTPC3U': -0.4375,
   'A2IFGGXG3YV3Y6': 0.5625,
   'A2LW5AL0KQ9P1M': 0.5625,
   'A3BVN2QBNPBHCR': 0.5625,
   'ACJT8MUC0LRF0': 0.5625,
   'AG1OL1K3SFURN

In [0]:
rdd4.take(3)

[('B009APQ6S6',
  [('ACJT8MUC0LRF0', 0.33333333333333304),
   ('A225G2TFM76GYX', 0.33333333333333304),
   ('ARADXCOUVBS09', -0.666666666666667),
   ('AAEHOTZ1GRAUX', -0.666666666666667),
   ('A2LW5AL0KQ9P1M', 0.33333333333333304),
   ('A7H1LWFBE905R', 0.33333333333333304)],
  11.489125293076057),
 ('B000T9LUB0',
  [('A2BVNVJOFXGZUB', -1.5555555555555554),
   ('A225G2TFM76GYX', -1.5555555555555554),
   ('A8G2W3PNBM0R2', 2.4444444444444446),
   ('A2QQEURPB1VQF7', -1.5555555555555554),
   ('A1PCYAHFDDMGJZ', -1.5555555555555554),
   ('A250AXLRBVYKB4', -0.5555555555555554),
   ('A2I3XS9TO93Q0F', 2.4444444444444446),
   ('A2UOHALGF2X77Q', -0.5555555555555554),
   ('A1H378KKY8V01U', 2.4444444444444446)],
  9.327379053088816),
 ('B001U3PYLQ',
  [('A2BVNVJOFXGZUB', -0.08000000000000007),
   ('AJK49DENMS2BT', -3.08),
   ('AFBXQDFJF33VS', -0.08000000000000007),
   ('A57Q87B0TC9IJ', -0.08000000000000007),
   ('A1FRADBDXQPJJC', 0.9199999999999999),
   ('A1WSFHRBY2ZD1R', 0.9199999999999999),
   ('A1

In [0]:
def sim(x):
  dict1={}
  dict2={}
  for rec in recNeed.value:
    dict1[rec[0]]=0
    dict2[rec[0]]=0
  for tp in x[1]:
    for rec in recNeed.value:
      if tp[0] in rec[1]:
        dict1[rec[0]]+=tp[1]*rec[1][tp[0]]
        dict2[rec[0]]+=1
  for rec in recNeed.value:
    if dict2[rec[0]]>=2:
      dict1[rec[0]]= dict1[rec[0]]/(rec[2]*x[2])
    else:
       dict1[rec[0]]=0
  op=[]
  for k in dict1:
    op.append((k,[(x[0],dict1[k])]))
  return op

In [0]:
def getTopNeighbours(x):
  return (x[0],sorted(x[1],key=(lambda k:k[1]),reverse=True)[:50])

In [0]:
rdd4.map(lambda x:sim(x)).take(5)

[[('0321719816', [('B009APQ6S6', 0)]),
  ('B00JHN1MJO', [('B009APQ6S6', 0.0011167838904560354)])],
 [('0321719816', [('B000T9LUB0', 0)]), ('B00JHN1MJO', [('B000T9LUB0', 0)])],
 [('0321719816', [('B001U3PYLQ', 0.0009874768872348387)]),
  ('B00JHN1MJO', [('B001U3PYLQ', 0)])],
 [('0321719816', [('B004DTLWZ6', 0)]), ('B00JHN1MJO', [('B004DTLWZ6', 0)])],
 [('0321719816', [('B00843U4I8', 0)]), ('B00JHN1MJO', [('B00843U4I8', 0)])]]

In [0]:
rdd4.flatMap(lambda x:sim(x)).filter(lambda x: x[1][0][1]>0).collect()

[('B00JHN1MJO', [('B009APQ6S6', 0.0011167838904560354)]),
 ('0321719816', [('B001U3PYLQ', 0.0009874768872348387)]),
 ('0321719816', [('0321719824', 0.002095037539246224)]),
 ('B00JHN1MJO', [('B00EOI2SR2', 0.00023908258015638764)]),
 ('B00JHN1MJO', [('B00HRQB28Y', 0.00017678842800891218)]),
 ('B00JHN1MJO', [('1413313728', 0.0009718403747841316)]),
 ('B00JHN1MJO', [('B009ENDZ4C', 0.0017776063021535658)]),
 ('B00JHN1MJO', [('B011DMUIQY', 0.004332961966944902)]),
 ('0321719816', [('B000Y7Y6IQ', 0.007684387618065217)]),
 ('B00JHN1MJO', [('B00IIL0S8W', 0.0014788325345356224)]),
 ('B00JHN1MJO', [('B00JC5Y6YA', 0.0019415581015209352)]),
 ('B00JHN1MJO', [('B00N4OLGL6', 0.0030961140540887466)]),
 ('0321719816', [('B005LXIJ7M', 0.001059838490165782)]),
 ('B00JHN1MJO', [('B005LXIJ7M', 0.0030474700652593518)]),
 ('B00JHN1MJO', [('B00CNLGONQ', 0.001837290845528607)]),
 ('B00JHN1MJO', [('B00F9ZQQ8Q', 0.005229481130840864)]),
 ('B00JHN1MJO', [('B000XHE0Q0', 0.0016062587819664531)]),
 ('B00JHN1MJO', [(

In [0]:
def formatter_misc(x):
  dict1={}
  for i in x[1]:
    dict1[i[0]]=i[1]
  return (x[0],dict1)

In [0]:
tempNeed=sc.broadcast(rdd4.flatMap(lambda x:sim(x)).filter(lambda x: x[1][0][1]>0).reduceByKey(lambda a,b: a+b).map(lambda x: getTopNeighbours(x)).map(lambda x: formatter_misc(x)).collect())

In [0]:
itemset=set()
dict1={}
for i in tempNeed.value:
  tset=set()
  for j in i[1]:
    dict1[(i[0],j)]=i[1][j]
    tset.add(j)
  itemset=itemset.union(tset)

In [0]:
itemNeed=sc.broadcast(itemset)
simNeed=sc.broadcast(dict1)

In [0]:
simNeed.value

{('0321719816', '0321719824'): 0.002095037539246224,
 ('0321719816', 'B000Y7Y6IQ'): 0.007684387618065217,
 ('0321719816', 'B001U3PYLQ'): 0.0009874768872348387,
 ('0321719816', 'B003YJ8WFE'): 0.008361116836464635,
 ('0321719816', 'B005LXIJ7M'): 0.001059838490165782,
 ('0321719816', 'B009716H00'): 0.0034887169858817107,
 ('B00JHN1MJO', '1413313728'): 0.0009718403747841316,
 ('B00JHN1MJO', 'B000XHE0Q0'): 0.0016062587819664531,
 ('B00JHN1MJO', 'B002PDPIJI'): 0.0008011718070653781,
 ('B00JHN1MJO', 'B0042XZ1I2'): 0.00030311133950410723,
 ('B00JHN1MJO', 'B005LXIJ7M'): 0.0030474700652593518,
 ('B00JHN1MJO', 'B008414ZYY'): 0.001033786937717645,
 ('B00JHN1MJO', 'B009APQ6S6'): 0.0011167838904560354,
 ('B00JHN1MJO', 'B009ENDZ4C'): 0.0017776063021535658,
 ('B00JHN1MJO', 'B00CNLGONQ'): 0.001837290845528607,
 ('B00JHN1MJO', 'B00EOI2SR2'): 0.00023908258015638764,
 ('B00JHN1MJO', 'B00F9ZQQ8Q'): 0.005229481130840864,
 ('B00JHN1MJO', 'B00G6CLN3Y'): 0.008000917707396998,
 ('B00JHN1MJO', 'B00HRQB28Y'): 0.0

In [0]:
def calc(x):
  op=[]
  for k in ['0321719816','B00JHN1MJO']:
    if (k,x[0]) in simNeed.value:
      for i in x[1]:
        op.append(((k,i[0]),(i[1]*simNeed.value[(k,x[0])],simNeed.value[(k,x[0])],1)))
  return op

In [0]:
recNeed.value

[('0321719816',
  {'A14638TGYH7GD9': 0.4285714285714288,
   'A14JBDSWKPKTZA': 0.4285714285714288,
   'A2BVNVJOFXGZUB': 0.4285714285714288,
   'A2FRAS05LEI6JJ': -0.5714285714285712,
   'A2JMJVNTBL7K7E': 0.4285714285714288,
   'A2VWE5SGNDS8HW': -1.5714285714285712,
   'A2WCFDOCS73PNB': 0.4285714285714288,
   'A31ICLWQ9CSHRS': 0.4285714285714288,
   'A38NELQT98S4H8': -0.5714285714285712,
   'A3M1PA1C98XNGN': -0.5714285714285712,
   'A3QJU4FEN8PQSZ': -0.5714285714285712,
   'ACJT8MUC0LRF0': 0.4285714285714288,
   'AMRMK86X3PKXD': 0.4285714285714288,
   'AYUF7YETYOLNX': 0.4285714285714288},
  17.26267650163207),
 ('B00JHN1MJO',
  {'A10G4BPT5MGBHY': -0.4375,
   'A1BEKAQQGP8CMF': -0.4375,
   'A1GWG5CWLKJ7ET': 0.5625,
   'A1HOLE9R6WPT85': 0.5625,
   'A1V3TRGWOMA8LC': 0.5625,
   'A225G2TFM76GYX': -0.4375,
   'A25NT9MZV9RTR7': -0.4375,
   'A26R7LLAFTPC3U': -0.4375,
   'A2IFGGXG3YV3Y6': 0.5625,
   'A2LW5AL0KQ9P1M': 0.5625,
   'A3BVN2QBNPBHCR': 0.5625,
   'ACJT8MUC0LRF0': 0.5625,
   'AG1OL1K3SFURN

In [0]:
dict1={}
for i in recNeed.value:
    dict1[i[0]]=i[1]
dict1['0321719816']

{'A14638TGYH7GD9': 0.4285714285714288,
 'A14JBDSWKPKTZA': 0.4285714285714288,
 'A2BVNVJOFXGZUB': 0.4285714285714288,
 'A2FRAS05LEI6JJ': -0.5714285714285712,
 'A2JMJVNTBL7K7E': 0.4285714285714288,
 'A2VWE5SGNDS8HW': -1.5714285714285712,
 'A2WCFDOCS73PNB': 0.4285714285714288,
 'A31ICLWQ9CSHRS': 0.4285714285714288,
 'A38NELQT98S4H8': -0.5714285714285712,
 'A3M1PA1C98XNGN': -0.5714285714285712,
 'A3QJU4FEN8PQSZ': -0.5714285714285712,
 'ACJT8MUC0LRF0': 0.4285714285714288,
 'AMRMK86X3PKXD': 0.4285714285714288,
 'AYUF7YETYOLNX': 0.4285714285714288}

In [0]:
def shuffleCust(x):
  return (x[0][0],[(x[0][1],x[1])])

def compare(x):
  op=set()
  dict1={}
  for i in recNeed.value:
    dict1[i[0]]=i[1]
  for i in x[1]:
    if i[0] in dict1[x[0]]:
      op.add((i[0],dict1[x[0]][i[0]]))
    else:
      op.add((i[0],i[1]))

  for i in dict1[x[0]]:
    op.add((i,dict1[x[0]][i]))

  return (x[0],list(op))

In [0]:
rdd4.filter(lambda x: x[0] in itemNeed.value).flatMap(lambda x:calc(x)).take(4)

[(('B00JHN1MJO', 'ACJT8MUC0LRF0'),
  (0.0003722612968186781, 0.0011167838904560354, 1)),
 (('B00JHN1MJO', 'A225G2TFM76GYX'),
  (0.0003722612968186781, 0.0011167838904560354, 1)),
 (('B00JHN1MJO', 'ARADXCOUVBS09'),
  (-0.0007445225936373572, 0.0011167838904560354, 1)),
 (('B00JHN1MJO', 'AAEHOTZ1GRAUX'),
  (-0.0007445225936373572, 0.0011167838904560354, 1))]

In [0]:
rdd5=rdd4.filter(lambda x: x[0] in itemNeed.value).flatMap(lambda x:calc(x)).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1],a[2]+b[2])).filter(lambda x: x[1][2]>1).map(lambda x:(x[0],x[1][0]/x[1][1]))


In [0]:
rdd5.map(lambda x: shuffleCust(x)).reduceByKey(lambda a,b:a+b).map(lambda x:compare(x)).collect()

[('B00JHN1MJO',
  [('ACJT8MUC0LRF0', 0.5625),
   ('A18YMFFJW974QS', 0.9413824659303727),
   ('A7H1LWFBE905R', -0.47024802107399377),
   ('A3VI2VETB90ZG5', 0.791612666112398),
   ('A3BVN2QBNPBHCR', 0.5625),
   ('A5KMMY627T3W', 0.6737346285032605),
   ('A3FEGTOLCWXSV4', 0.4319313946607017),
   ('A26R7LLAFTPC3U', -0.4375),
   ('A1HOLE9R6WPT85', 0.5625),
   ('A3SX5MZTCZRQLF', 0.8568435727492163),
   ('A2T7GZ74MZI0MN', 0.5760513333963343),
   ('A1GYEGLX3P2Y7P', -0.06537831912854628),
   ('A3TAS1AG6FMBQW', 0.5368969855257552),
   ('A105S56ODHGJEK', -0.33375454635645674),
   ('A1GWG5CWLKJ7ET', 0.5625),
   ('A1B70ZEWQ6UH1A', 0.5327496738087129),
   ('A3FOSLM5G1BO63', -0.5092893881256859),
   ('A1V3TRGWOMA8LC', 0.5625),
   ('A3P8ZDHC7XSKSE', -1.0914152105029258),
   ('A2E1EFNIZL2FVA', 0.7158473439544502),
   ('AUV3OR951650C', 1.3099037581786168),
   ('A3RGHO0Q938FRS', 1.3594077752069225),
   ('A1GQRGB8FGSLIZ', -0.22587136746046368),
   ('ACVDL5QRCOPE1', 0.7761142461634319),
   ('A14NA0W8ESGDSI'

In [0]:
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################

In [0]:
iprdd=sc.textFile('/content/drive/My Drive/BigDataA1/Software_5.json',4)


In [0]:
import json
import re
import numpy as np
def getWordCount(x):
  curr=json.loads(x)
  op=[]
  try:
    li=re.findall( r'((?:[\.,!?;"])|(?:(?:\#|\@)?[A-Za-z0-9_\-]+(?:\'[a-z]{1,3})?))',curr['reviewText'].lower())
    for i in li: 
      op.append((i,1))
  except:
    pass
  return op


12805

In [0]:
top_thou=iprdd.flatMap(lambda x: getWordCount(x)).reduceByKey(lambda a,b: a+b).map(lambda x: (x[1],x[0])).sortByKey(ascending=False).map(lambda x: x[1]).take(1000)
bctop=sc.broadcast(set(top_thou))

In [0]:
def getExtras(x):
  curr=json.loads(x)
  op=[]
  try:
    li=re.findall( r'((?:[\.,!?;"])|(?:(?:\#|\@)?[A-Za-z0-9_\-]+(?:\'[a-z]{1,3})?))',curr['reviewText'].lower())
    ipset=set(li)
    k=bctop.value-ipset
    for i in k:
      p= 1 if curr['verified'] or curr['verified']=='True' else 0
      op.append(((i,curr['overall'],p),1))
  except:
    pass
  return op


In [0]:
extras_list=iprdd.flatMap(lambda x:getExtras(x)).reduceByKey(lambda a,b:a+b).map(lambda x: (x[0][0],[x])).reduceByKey(lambda a,b: a+b).collect()
bcExtras=sc.broadcast(extras_list)

In [0]:
dict1={}
for i in bcExtras.value:
  dict1[i[0]]=i[1]
bcExtras=sc.broadcast(dict1)

In [0]:
bcExtras.value

{'was': [(('was', 3.0, 0), 560),
  (('was', 4.0, 1), 606),
  (('was', 5.0, 1), 2562),
  (('was', 1.0, 1), 273),
  (('was', 2.0, 0), 233),
  (('was', 5.0, 0), 1563),
  (('was', 1.0, 0), 583),
  (('was', 2.0, 1), 133),
  (('was', 4.0, 0), 1081),
  (('was', 3.0, 1), 300)],
 'mobile': [(('mobile', 3.0, 0), 1104),
  (('mobile', 4.0, 1), 829),
  (('mobile', 5.0, 1), 3248),
  (('mobile', 1.0, 1), 426),
  (('mobile', 2.0, 0), 505),
  (('mobile', 5.0, 0), 2672),
  (('mobile', 1.0, 0), 1065),
  (('mobile', 2.0, 1), 205),
  (('mobile', 4.0, 0), 2120),
  (('mobile', 3.0, 1), 439)],
 'tools': [(('tools', 3.0, 0), 1054),
  (('tools', 4.0, 1), 819),
  (('tools', 5.0, 1), 3223),
  (('tools', 1.0, 1), 423),
  (('tools', 2.0, 0), 488),
  (('tools', 5.0, 0), 2427),
  (('tools', 1.0, 0), 1059),
  (('tools', 2.0, 1), 204),
  (('tools', 4.0, 0), 1933),
  (('tools', 3.0, 1), 441)],
 'task': [(('task', 3.0, 0), 1104),
  (('task', 4.0, 1), 827),
  (('task', 5.0, 1), 3241),
  (('task', 1.0, 1), 423),
  (('task'

In [0]:
def getMatchCount(x):
  curr=json.loads(x)
  op=[]
  dict1={}
  try:
    li=re.findall( r'((?:[\.,!?;"])|(?:(?:\#|\@)?[A-Za-z0-9_\-]+(?:\'[a-z]{1,3})?))',curr['reviewText'].lower())
    tot=len(li)
    for i in li: 
      if i in bctop.value and i in dict1:
        dict1[i]+=1
      elif i in bctop.value:
        dict1[i]=1

    for i in dict1:
      p=1 if curr['verified'] or curr['verified']=='True' else 0
      op.append((i,[[curr['overall'],dict1[i]/tot,p]]))
    
  except:
    pass

  return op

In [0]:
def getRemCount(x):
  curr=json.loads(x)
  setx=set()
  try:
    li=re.findall( r'((?:[\.,!?;"])|(?:(?:\#|\@)?[A-Za-z0-9_\-]+(?:\'[a-z]{1,3})?))',curr['reviewText'].lower())
    for i in li: 
      if i in bctop.value:
        setx.add(i)
    
  except:
    pass

  return (curr['overall'],len(bctop.value)-len(setx))

In [0]:

trdd=iprdd.flatMap(lambda x:getMatchCount(x)).reduceByKey(lambda a,b:a+b)

In [0]:
tli=trdd.take(1)

In [0]:
tli

[('using',
  [[4.0, 0.004201680672268907, 0],
   [4.0, 0.001949317738791423, 0],
   [5.0, 0.0024183796856106408, 0],
   [5.0, 0.004149377593360996, 0],
   [5.0, 0.0029585798816568047, 0],
   [5.0, 0.0045662100456621, 0],
   [3.0, 0.003816793893129771, 0],
   [4.0, 0.0011441647597254005, 0],
   [4.0, 0.014285714285714285, 0],
   [1.0, 0.004424778761061947, 0],
   [1.0, 0.006172839506172839, 1],
   [5.0, 0.007874015748031496, 1],
   [4.0, 0.0033975084937712344, 1],
   [4.0, 0.01639344262295082, 0],
   [3.0, 0.002044989775051125, 0],
   [4.0, 0.004629629629629629, 0],
   [4.0, 0.0014360938247965534, 0],
   [5.0, 0.006211180124223602, 0],
   [5.0, 0.0036900369003690036, 0],
   [3.0, 0.005780346820809248, 0],
   [5.0, 0.0017513134851138354, 0],
   [3.0, 0.0018484288354898336, 0],
   [5.0, 0.0009900990099009901, 0],
   [5.0, 0.007407407407407408, 0],
   [4.0, 0.0016155088852988692, 0],
   [4.0, 0.0078125, 0],
   [3.0, 0.0014388489208633094, 0],
   [4.0, 0.019867549668874173, 0],
   [4.0, 0.0

In [0]:
bcExtras.value

In [0]:
import numpy as np
from scipy.stats import t

def getBetaAndPval(x):
  tup_arr=np.asarray(x[1])
  tup_arr=tup_arr.transpose()
  Y=np.asarray(tup_arr[0])
  X=None
  XC=None
  YC=None
  isVerfiedConsidered=True
  dff=0
  cons_mean=0
  if isVerfiedConsidered:
    X=np.asarray(tup_arr[1:3])
    X=X.transpose()
    li=bcExtras.value[x[0]][-0.18015738770120795, 0.2097298456670961]
    for i in li:
      k=[i[0][1]]*i[1]
      l=[[0,i[0][2]]]*i[1]
      
      Y=np.concatenate([Y,np.asarray(k)])
      X=np.concatenate([X,np.asarray(l)])
    
    XC=X.copy()
    YC=Y.copy()
    Xtrans=X.transpose()
    for j in range(len(Xtrans)):
      xmean=np.mean(Xtrans[j])
      if j==0:
        cons_mean=xmean
      xstd=np.std(Xtrans[j])
      Xtrans[j]=(Xtrans[j]-xmean)/xstd

    X=Xtrans.transpose()

    ymean=np.mean(Y)
    ystd=np.std(Y)
    Y=(Y-ymean)/ystd
    X_trans=Xtrans
    sq=X_trans.dot(X)
    beta=(np.linalg.inv(sq)).dot(X_trans.dot(Y))
    dff=len(X)-3


  else:
    X=np.asarray(tup_arr[1],)
    li=bcExtras.value[x[0]]
    for i in li:
      k=[i[0][1]]*i[1]
      X=np.concatenate([X,np.asarray([0]*i[1])])
      Y=np.concatenate([Y,np.asarray(k)])
    
    XC=X.reshape(len(X),1).copy()
    YC=Y.copy()
    xmean=np.mean(X)
    cons_mean=xmean
    ymean=np.mean(Y)
    xstd=np.std(X)
    ystd=np.std(Y)
    X=(X-xmean)/xstd
    Y=(Y-ymean)/ystd
    X=X.reshape(len(X),1)
    X_trans=X.transpose()
    sq=X_trans.dot(X)
    beta=(np.linalg.inv(sq)).dot(X_trans.dot(Y))
    dff=len(X)-2

 
  Ycap=X.dot(beta)
  
  rss=0
  for i in range(len(Ycap)):
    rss+=(Ycap[i]-Y[i])**2
  
  betaj=beta[0]
  x_sq_sum=0
  cons_mean=np.mean(X.transpose()[0])
  for i in X.transpose()[0]:
    x_sq_sum+=(i-cons_mean)**2
  tval=float(betaj/(((rss/dff)/x_sq_sum)**0.5))
  if betaj < 0 :
    pval=t.cdf(tval,dff)*1000
  else:
    pval=float(float(1)-float(t.cdf(tval,dff)))*1000
  return (x[0],beta.tolist(),pval)


In [0]:
t.ppf(0.556,12000)

0.14083836269727104

In [0]:
getBetaAndPval(tli[0])

('using', [0.06004427150225921, 0.2014612963482425], 1.8848256289061283e-09)

In [0]:

  Ycap=X.dot(beta)
  rss=0
  for i in range(len(Ycap)):
    rss=(Ycap[i]-Y[i])**2
  s_square=rss/dff
  print(s_square)
  betaj=beta[0]
  x_sq_sum=0
  for i in X.transpose()[0]:
    x_sq_sum+=(i-cons_mean)**2
  print(x_sq_sum,s_square)
  den=(s_square/x_sq_sum)**0.5
  print(den)
  tval=betaj/den
  print(tval)
  print(t.ppf(0.01,dff)*1000)

In [0]:
cRdd=trdd.map(lambda x: getBetaAndPval(x)).map(lambda x:(x[1][0],x))
li1=cRdd.sortByKey(ascending=True).map(lambda x:x[1]).take(20)
li2=cRdd.sortByKey(ascending=False).map(lambda x:x[1]).take(20)
li=li1+li2
li

[('not', [-0.18015738770120795, 0.2097298456670961], 3.339445408694214e-95),
 ('support', [-0.1073383327743252, 0.2029703708363923], 7.718278815490548e-33),
 ('they', [-0.10631066948947034, 0.20135631609370055], 3.437705697861711e-32),
 ('intuit',
  [-0.10388553259099051, 0.2062192613366757],
  1.1009334181072996e-30),
 ('error', [-0.09513216708574958, 0.2025558349674489], 1.529259506347534e-25),
 ('trying', [-0.08226541278765123, 0.2001615535876175], 8.346966814477405e-19),
 ('their', [-0.0813892287852031, 0.2001907029933036], 2.2123758910724338e-18),
 ('bad', [-0.07996675449737177, 0.20681341102979617], 1.0531081522616334e-17),
 ('back', [-0.07985537691417421, 0.20254487804062352], 1.1882091193449436e-17),
 ('pay', [-0.07908535106525755, 0.20449305747710184], 2.7307840215605874e-17),
 ('message',
  [-0.0786215418665852, 0.20338491599542077],
  4.490567858463664e-17),
 ('crashes',
  [-0.0781130596208113, 0.20036624589533028],
  7.725067265768298e-17),
 ('uninstall',
  [-0.077896581550

In [0]:
li

[('not', [-0.17328571875402213], nan),
 ('they', [-0.11099042513659156], nan),
 ('support', [-0.10895835650949082], nan),
 ('intuit', [-0.09913694324819351], nan),
 ('that', [-0.0987075463597484], nan),
 ('error', [-0.09783895038250721], nan),
 ('trying', [-0.0911841241924111], nan),
 ('their', [-0.0903335681602523], nan),
 ('vista', [-0.08860984894986987], nan),
 ('crashes', [-0.08699042735569065], nan),
 ('uninstall', [-0.08688119732628827], nan),
 ('back', [-0.08310760000972331], nan),
 ("won't", [-0.08251565322473489], nan),
 ('if', [-0.08083148330076147], nan),
 ('off', [-0.08056660202720958], nan),
 ('message', [-0.0797654213333899], nan),
 ('hours', [-0.07940766871885706], nan),
 ('told', [-0.0793716416371756], nan),
 ('fix', [-0.07840190911361496], nan),
 ('pay', [-0.07736336756284996], nan),
 ('easy', [0.11981643995066478], nan),
 ('great', [0.0957183348426632], nan),
 ('and', [0.09559168720195188], nan),
 ('years', [0.08156362567548736], nan),
 ('very', [0.07409137080167172],